## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,0,Atuona,-9.8000,-139.0333,78.04,75,51,15.79,PF,broken clouds
1,1,Jamestown,42.0970,-79.2353,76.59,50,40,15.99,US,scattered clouds
2,2,Provideniya,64.3833,-173.3000,42.67,80,99,10.54,RU,light rain
3,3,Klaksvik,62.2266,-6.5890,50.92,80,89,6.44,FO,overcast clouds
4,4,Hermanus,-34.4187,19.2345,57.90,87,99,8.30,ZA,overcast clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,0,Atuona,-9.8000,-139.0333,78.04,75,51,15.79,PF,broken clouds
1,1,Jamestown,42.0970,-79.2353,76.59,50,40,15.99,US,scattered clouds
13,13,Puerto Ayora,-0.7393,-90.3518,76.96,85,99,8.01,EC,overcast clouds
15,15,Manggar,-2.8833,108.2667,81.00,81,100,8.77,ID,light rain
20,20,Avarua,-21.2078,-159.7750,77.05,83,20,12.66,CK,few clouds
23,23,Mankato,44.1636,-93.9994,84.96,65,1,0.00,US,clear sky
28,28,Butaritari,3.0707,172.7902,82.94,76,17,19.48,KI,light rain
29,29,Huaicheng,23.9051,112.1931,79.99,79,2,1.50,CN,light rain
37,37,Albany,42.6001,-73.9662,77.14,49,23,7.94,US,few clouds
39,39,Bengkulu,-3.8004,102.2655,75.20,93,99,5.75,ID,overcast clouds


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

Unnamed: 0             224
City                   224
Lat                    224
Lng                    224
Max Temp               224
Humidity               224
Cloudiness             224
Wind Speed             224
Country                224
Current Description    224
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Atuona,PF,78.04,-9.8000,-139.0333,
1,Jamestown,US,76.59,42.0970,-79.2353,
13,Puerto Ayora,EC,76.96,-0.7393,-90.3518,
15,Manggar,ID,81.00,-2.8833,108.2667,
20,Avarua,CK,77.05,-21.2078,-159.7750,
23,Mankato,US,84.96,44.1636,-93.9994,
28,Butaritari,KI,82.94,3.0707,172.7902,
29,Huaicheng,CN,79.99,23.9051,112.1931,
37,Albany,US,77.14,42.6001,-73.9662,
39,Bengkulu,ID,75.20,-3.8004,102.2655,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
    }

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
       
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"  

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()

    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        pass

# len(hotels["results"])


Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df = hotel_df[hotel_df['Hotel Name'] != '']
hotel_df

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Atuona,PF,78.04,-9.8000,-139.0333,Villa Enata
1,Jamestown,US,76.59,42.0970,-79.2353,Hampton Inn & Suites Jamestown
13,Puerto Ayora,EC,76.96,-0.7393,-90.3518,Finch Bay Galapagos Hotel
15,Manggar,ID,81.00,-2.8833,108.2667,Guest Hotel
20,Avarua,CK,77.05,-21.2078,-159.7750,Paradise Inn
...,...,...,...,...,...,...
649,Dongli,CN,82.38,23.5656,116.8194,Chenghai Shizhang Donghuaqiao Hostel
650,Kailua,US,82.38,21.4022,-157.7394,Sheffield House Bed & Breakfast
653,Bensonville,LR,76.71,6.4461,-10.6125,Moinga
656,Nador,MA,75.27,35.1740,-2.9287,Hôtel La Giralda


In [11]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<d1>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>Current Description</dd>
<dt>Max Temp</dt><dd>{Max Temp}</dd>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
                                max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))